# Description

There are 4 individual sets of data that are considered namely:

1. dataset_62344690_fitbit_intraday_steps_df (Fitbit Intraday steps)
2. dataset_62344690_fitbit_sleep_daily_summary_df (Fitbit Sleep - Daily summary)
3. dataset_62344690_fitbit_sleep_level_df (Fitbit Sleep - Level) <b><u>Note: IGNORING SLEEP LEVEL</u></b>
4. dataset_62344690_survey_df (Overall Health survey questionnaire)

In this notebook, each of these dataframes is saved individually

In [1]:
# Imports

import pandas as pd
import numpy as np
import os

In [2]:
dataset = os.getenv('WORKSPACE_CDR')
my_bucket = os.getenv('WORKSPACE_BUCKET')

# Saving Fitbit Intraday Steps locally

In [ ]:
%%time

# This query represents dataset "oh_dataset" for domain "fitbit_intraday_steps" and was generated for All of Us Registered Tier Dataset v7
dataset_62344690_fitbit_intraday_steps_sql = """
    SELECT
        steps_intraday.person_id,
        CAST(steps_intraday.datetime AS DATE) as date,
        SUM(CAST(steps_intraday.steps AS INT64)) as sum_steps 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.steps_intraday` steps_intraday   
    WHERE
        steps_intraday.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE,dob, YEAR) - IF(EXTRACT(MONTH 
                    FROM
                        dob)*100 + EXTRACT(DAY 
                    FROM
                        dob) > EXTRACT(MONTH 
                    FROM
                        CURRENT_DATE)*100 + EXTRACT(DAY 
                    FROM
                        CURRENT_DATE),
                        1,
                        0) BETWEEN 18 AND 120 
                        AND NOT EXISTS ( SELECT
                            'x' 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d 
                        WHERE
                            d.person_id = p.person_id) ) 
                        AND cb_search_person.person_id IN (SELECT
                            person_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                        WHERE
                            has_fitbit_sleep_daily_summary = 1 ) 
                        AND cb_search_person.person_id IN (SELECT
                            person_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                        WHERE
                            has_fitbit_sleep_level = 1 ) 
                        AND cb_search_person.person_id IN (SELECT
                            person_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                        WHERE
                            has_fitbit_steps_intraday = 1 ) 
                        AND cb_search_person.person_id IN (SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (
                                        SELECT
                                            DISTINCT c.concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                        JOIN
                                            (
                                                select
                                                    cast(cr.id as string) as id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                WHERE
                                                    concept_id IN (1585710) 
                                                    AND full_text LIKE '%_rank1]%'
                                            ) a 
                                                ON (
                                                    c.path LIKE CONCAT('%.',
                                                a.id,
                                                '.%') 
                                                OR c.path LIKE CONCAT('%.',
                                                a.id) 
                                                OR c.path LIKE CONCAT(a.id,
                                                '.%') 
                                                OR c.path = a.id) 
                                            WHERE
                                                is_standard = 0 
                                                AND is_selectable = 1
                                            ) 
                                            AND is_standard = 0 
                                    )
                                ) criteria 
                            ) ) 
                    GROUP BY
                        person_id,
                        date"""

fitbit_intraday_steps_df = pd.read_gbq(
    dataset_62344690_fitbit_intraday_steps_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

In [4]:
# Saving locally

file_name = "fitbit_intraday_steps.csv"
fitbit_intraday_steps_df.to_csv(f"{file_name}", index=False)

# Saving Fitbit Sleep Daily Summary locally

In [5]:
%%time

# This query represents dataset "oh_dataset" for domain "fitbit_sleep_daily_summary" and was generated for All of Us Registered Tier Dataset v7
dataset_62344690_fitbit_sleep_daily_summary_sql = """
    SELECT
        sleep_daily_summary.person_id,
        sleep_daily_summary.sleep_date,
        sleep_daily_summary.is_main_sleep,
        sleep_daily_summary.minute_in_bed,
        sleep_daily_summary.minute_asleep,
        sleep_daily_summary.minute_after_wakeup,
        sleep_daily_summary.minute_awake,
        sleep_daily_summary.minute_restless,
        sleep_daily_summary.minute_deep,
        sleep_daily_summary.minute_light,
        sleep_daily_summary.minute_rem,
        sleep_daily_summary.minute_wake 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.sleep_daily_summary` sleep_daily_summary   
    WHERE
        PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE,dob, YEAR) - IF(EXTRACT(MONTH 
                    FROM
                        dob)*100 + EXTRACT(DAY 
                    FROM
                        dob) > EXTRACT(MONTH 
                    FROM
                        CURRENT_DATE)*100 + EXTRACT(DAY 
                    FROM
                        CURRENT_DATE),
                        1,
                        0) BETWEEN 18 AND 120 
                        AND NOT EXISTS ( SELECT
                            'x' 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d 
                        WHERE
                            d.person_id = p.person_id) ) 
                        AND cb_search_person.person_id IN (SELECT
                            person_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                        WHERE
                            has_fitbit_sleep_daily_summary = 1 ) 
                        AND cb_search_person.person_id IN (SELECT
                            person_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                        WHERE
                            has_fitbit_sleep_level = 1 ) 
                        AND cb_search_person.person_id IN (SELECT
                            person_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                        WHERE
                            has_fitbit_steps_intraday = 1 ) 
                        AND cb_search_person.person_id IN (SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (
                                        SELECT
                                            DISTINCT c.concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                        JOIN
                                            (
                                                select
                                                    cast(cr.id as string) as id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                WHERE
                                                    concept_id IN (1585710) 
                                                    AND full_text LIKE '%_rank1]%'
                                            ) a 
                                                ON (
                                                    c.path LIKE CONCAT('%.',
                                                a.id,
                                                '.%') 
                                                OR c.path LIKE CONCAT('%.',
                                                a.id) 
                                                OR c.path LIKE CONCAT(a.id,
                                                '.%') 
                                                OR c.path = a.id) 
                                            WHERE
                                                is_standard = 0 
                                                AND is_selectable = 1
                                            ) 
                                            AND is_standard = 0 
                                    )
                                ) criteria 
                            ) )"""

fitbit_sleep_daily_summary_df = pd.read_gbq(
    dataset_62344690_fitbit_sleep_daily_summary_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

Downloading:   0%|          | 0/14297905 [00:00<?, ?rows/s]

CPU times: user 1min 29s, sys: 8.33 s, total: 1min 38s
Wall time: 1min 46s


In [6]:
# Saving locally

file_name = "fitbit_sleep_daily_summary.csv"
fitbit_sleep_daily_summary_df.to_csv(f"{file_name}", index=False)

# Saving Fitbit Sleep Level locally

# UPDATE: Ignoring sleep level due to high dimensionality

In [10]:
%%time

# This query represents dataset "oh_dataset" for domain "fitbit_sleep_level" and was generated for All of Us Registered Tier Dataset v7
# dataset_62344690_fitbit_sleep_level_sql = """
#     SELECT
#         sleep_level.person_id,
#         sleep_level.sleep_date,
#         sleep_level.is_main_sleep,
#         sleep_level.level,
#         CAST(sleep_level.start_datetime AS DATE) as date,
#         sleep_level.duration_in_min 
#     FROM
#         `""" + os.environ["WORKSPACE_CDR"] + """.sleep_level` sleep_level   
#     WHERE
#         PERSON_ID IN (
#             SELECT
#                 distinct person_id  
#             FROM
#                 `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
#             WHERE
#                 cb_search_person.person_id IN (
#                     SELECT
#                         person_id 
#                     FROM
#                         `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
#                     WHERE
#                         DATE_DIFF(CURRENT_DATE,dob, YEAR) - IF(EXTRACT(MONTH 
#                     FROM
#                         dob)*100 + EXTRACT(DAY 
#                     FROM
#                         dob) > EXTRACT(MONTH 
#                     FROM
#                         CURRENT_DATE)*100 + EXTRACT(DAY 
#                     FROM
#                         CURRENT_DATE),
#                         1,
#                         0) BETWEEN 18 AND 120 
#                         AND NOT EXISTS ( SELECT
#                             'x' 
#                         FROM
#                             `""" + os.environ["WORKSPACE_CDR"] + """.death` d 
#                         WHERE
#                             d.person_id = p.person_id) ) 
#                         AND cb_search_person.person_id IN (SELECT
#                             person_id 
#                         FROM
#                             `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
#                         WHERE
#                             has_fitbit_sleep_daily_summary = 1 ) 
#                         AND cb_search_person.person_id IN (SELECT
#                             person_id 
#                         FROM
#                             `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
#                         WHERE
#                             has_fitbit_sleep_level = 1 ) 
#                         AND cb_search_person.person_id IN (SELECT
#                             person_id 
#                         FROM
#                             `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
#                         WHERE
#                             has_fitbit_steps_intraday = 1 ) 
#                         AND cb_search_person.person_id IN (SELECT
#                             criteria.person_id 
#                         FROM
#                             (SELECT
#                                 DISTINCT person_id,
#                                 entry_date,
#                                 concept_id 
#                             FROM
#                                 `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
#                             WHERE
#                                 (
#                                     concept_id IN (
#                                         SELECT
#                                             DISTINCT c.concept_id 
#                                         FROM
#                                             `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
#                                         JOIN
#                                             (
#                                                 select
#                                                     cast(cr.id as string) as id 
#                                                 FROM
#                                                     `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
#                                                 WHERE
#                                                     concept_id IN (1585710) 
#                                                     AND full_text LIKE '%_rank1]%'
#                                             ) a 
#                                                 ON (
#                                                     c.path LIKE CONCAT('%.',
#                                                 a.id,
#                                                 '.%') 
#                                                 OR c.path LIKE CONCAT('%.',
#                                                 a.id) 
#                                                 OR c.path LIKE CONCAT(a.id,
#                                                 '.%') 
#                                                 OR c.path = a.id) 
#                                             WHERE
#                                                 is_standard = 0 
#                                                 AND is_selectable = 1
#                                             ) 
#                                             AND is_standard = 0 
#                                     )
#                                 ) criteria 
#                             ) )"""

# fitbit_sleep_level_df = pd.read_gbq(
#     dataset_62344690_fitbit_sleep_level_sql,
#     dialect="standard",
#     use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
#     progress_bar_type="tqdm_notebook")

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


In [11]:
# Saving locally

# file_name = "fitbit_sleep_level.csv"
# fitbit_sleep_level_df.to_csv(f"{file_name}", index=False)

In [12]:
# del fitbit_sleep_level_df

# Saving Survey locally

In [7]:
%%time

# This query represents dataset "oh_dataset" for domain "survey" and was generated for All of Us Registered Tier Dataset v7
dataset_62344690_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                SELECT
                    DISTINCT concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (
                        select
                            cast(cr.id as string) as id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                        WHERE
                            concept_id IN (
                                1585710
                            ) 
                            AND domain_id = 'SURVEY'
                    ) a 
                        ON (
                            c.path like CONCAT('%',
                        a.id,
                        '.%')) 
                    WHERE
                        domain_id = 'SURVEY' 
                        AND type = 'PPI' 
                        AND subtype = 'QUESTION'
                    )
            )  
            AND (
                answer.PERSON_ID IN (
                    SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                DATE_DIFF(CURRENT_DATE,dob, YEAR) - IF(EXTRACT(MONTH 
                            FROM
                                dob)*100 + EXTRACT(DAY 
                            FROM
                                dob) > EXTRACT(MONTH 
                            FROM
                                CURRENT_DATE)*100 + EXTRACT(DAY 
                            FROM
                                CURRENT_DATE),
                                1,
                                0) BETWEEN 18 AND 120 
                                AND NOT EXISTS ( SELECT
                                    'x' 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.death` d 
                                WHERE
                                    d.person_id = p.person_id) ) 
                                AND cb_search_person.person_id IN (SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    has_fitbit_sleep_daily_summary = 1 ) 
                                AND cb_search_person.person_id IN (SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    has_fitbit_sleep_level = 1 ) 
                                AND cb_search_person.person_id IN (SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    has_fitbit_steps_intraday = 1 ) 
                                AND cb_search_person.person_id IN (SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (
                                                SELECT
                                                    DISTINCT c.concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                JOIN
                                                    (
                                                        select
                                                            cast(cr.id as string) as id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                        WHERE
                                                            concept_id IN (1585710) 
                                                            AND full_text LIKE '%_rank1]%'
                                                    ) a 
                                                        ON (
                                                            c.path LIKE CONCAT('%.',
                                                        a.id,
                                                        '.%') 
                                                        OR c.path LIKE CONCAT('%.',
                                                        a.id) 
                                                        OR c.path LIKE CONCAT(a.id,
                                                        '.%') 
                                                        OR c.path = a.id) 
                                                    WHERE
                                                        is_standard = 0 
                                                        AND is_selectable = 1
                                                    ) 
                                                    AND is_standard = 0 
                                            )
                                        ) criteria 
                                    ) ))"""

survey_df = pd.read_gbq(
    dataset_62344690_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

Downloading:   0%|          | 0/259813 [00:00<?, ?rows/s]

CPU times: user 220 ms, sys: 101 ms, total: 321 ms
Wall time: 7.24 s


In [8]:
# Saving locally

file_name = "survey.csv"
survey_df.to_csv(f"{file_name}", index=False)